In [1]:
import numpy as np
import sys
import os
from typing import Sequence
import pandas as pd

sys.path.append(os.path.abspath("../libs"))
sys.path.append(os.path.abspath("../utils"))

from gradient_descendent import gradient_descendent
from plots.plots_tarefa1 import plot_function, plot_six_hump_camel

# Tarefa 1: Otimização unimodal sem restrição

## Parte 1: Otimização analítica

### Problema

Seja a função de custo quadrática dada por: 
$$J(x) = (x - c)^T A (x - c) + b$$
onde $A$ é uma matriz. Encontre analiticamente o ponto $x^* \in \mathbb{R}^2$ que minimize a função de custo.

### Solução

Temos que $\frac{\partial {((x - c)^T A (x - c))}}{\partial x} = (A^T + A)(x - c)$ 

ou 

$\frac{\partial {((x - c)^T A (x - c))}}{\partial x} = 2A(x - c)$ no caso de $A$ ser simétrica.

Portanto, derivando a função de custo e igualando a zero, temos:
$$J_x = (A^T + A)(x - c)$$

$$J_x = (A^T + A)(x - c) = 0$$

$$J_x = (A^T + A)^{-1}(A^T + A)(x - c) = (A^T + A)^{-1} \dot \quad 0$$

$$J_x = I(x - c) = (A^T + A)^{-1} \dot \quad 0$$

$$J_x = x - c = 0$$

$$J_x = x = c$$

$$x^* = c$$

$$J_{xx} = (A^T + A)$$

Assim, se $(A^T + A) > 0$, $x^* = c$ será um ponto de minímo.

No caso de a ser simétrica

$$J_x = 2A(x - c)$$

$$J_x = 2A(x - c) = 0$$

$$J_x = 2A^{-1}A(x - c) = A^{-1}\dot \quad 0$$

$$J_x = 2(x - c) = 0$$

$$J_x = x - c = 0$$

$$J_x = x = c$$

$$J_{xx} = 2A$$

Assim, se $2A > 0$, $x^* = c$ será um ponto de minímo.

## Parte 2: Otimização numérica por gradiente descendente

### Definir a função de custo

In [2]:
def wrapper(A: Sequence[Sequence[float | int]],
            b: float | int,
            c: Sequence[float | int]):
    A_np = np.array(A, dtype=np.float64)
    b_np = np.float64(b)
    c_np = np.array(c, dtype=np.float64)

    def cost_function(x: np.ndarray) -> float:
        x_np = np.array(x, dtype=np.float64)
        diff_x_c = x_np - c_np

        # (x - c)^T A (x - c) + b
        return float(diff_x_c.T @ A_np @ diff_x_c + b_np)

    def grad_function(x: np.ndarray) -> np.ndarray:
        x_np = np.array(x, dtype=np.float64)
        return (A_np.T + A_np) @ (x_np - c_np)

    return cost_function, grad_function

### Calcular o ponto mínimo usando gradiente descendente para diferentes taxas de aprendizado

In [3]:
A = [[4, 0],
     [1, 2]]
b = 2.0
c = [0, 1]

cost_fn, grad_cost_fn = wrapper(A, b, c)

x = [3, 3]
learning_rates = [0.1, 0.01, 0.001, 0.2]
max_iter = 1000
tolerance = 1e-6

dict_of_results = {}

for learning_rate in learning_rates:
     x_values, costs, num_iter = gradient_descendent(x, cost_fn, grad_cost_fn, learning_rate, max_iter, tolerance, 1)
     x_values = np.array(x_values)
     costs = np.array(costs)

     dict_of_results[learning_rate] = (x_values, costs)

     print(f"Taxa de aprendizado: {learning_rate}")
     print(f"Ponto mínimo: {x_values[-1]}, ou aproximadamente {np.round(x_values[-1], 4)}")
     print(f"Custo mínimo: {costs[-1]}, ou aproximadamente {np.round(costs[-1], 4)}")
     print(f"Numero de iterações: {num_iter}\n")
     print(f"{'-' * 100}\n")

Taxa de aprendizado: 0.1
Ponto mínimo: [-3.07360138e-08  1.00000013e+00], ou aproximadamente [-0.  1.]
Custo mínimo: 2.0000000000000338, ou aproximadamente 2.0
Numero de iterações: 34

----------------------------------------------------------------------------------------------------

Taxa de aprendizado: 0.01
Ponto mínimo: [-6.01402050e-08  1.00000025e+00], ou aproximadamente [-0.  1.]
Custo mínimo: 2.000000000000129, ou aproximadamente 2.0
Numero de iterações: 401

----------------------------------------------------------------------------------------------------

Taxa de aprendizado: 0.001
Ponto mínimo: [-0.00580973  1.02837632], ou aproximadamente [-0.0058  1.0284]
Custo mínimo: 2.001580583638381, ou aproximadamente 2.0016
Numero de iterações: 1000

----------------------------------------------------------------------------------------------------

Taxa de aprendizado: 0.2
Ponto mínimo: [-5.88840319e-08  9.99999986e-01], ou aproximadamente [-0.  1.]
Custo mínimo: 2.0000000000000

### Plotar a função de custo e o ponto mínimo encontrado para diferentes taxas de aprendizado

In [4]:
x_values, costs = dict_of_results[learning_rates[0]]

plot_function(cost_fn, x_values,costs, title=f"Superfície da Função de Custo + Pontos Encontrados (Taxa de Aprendizado = {learning_rates[0]})")

In [5]:
x_values, costs = dict_of_results[learning_rates[1]]

plot_function(cost_fn, x_values,costs, title=f"Superfície da Função de Custo + Pontos Encontrados (Taxa de Aprendizado = {learning_rates[1]})")

In [6]:
x_values, costs = dict_of_results[learning_rates[2]]

plot_function(cost_fn, x_values,costs, title=f"Superfície da Função de Custo + Pontos Encontrados (Taxa de Aprendizado = {learning_rates[2]})")

In [7]:
x_values, costs = dict_of_results[learning_rates[3]]

plot_function(cost_fn, x_values,costs, title=f"Superfície da Função de Custo + Pontos Encontrados (Taxa de Aprendizado = {learning_rates[3]})")

## Parte 3: Descida de gradiente aplicada a problemas multimodais

### Problema

Definir uma função de duas variáveis multimodal J(x) com, pelo menos, 3 minimos. Encontre o ponto mínimo usando gradiente descendente para diferentes taxas de aprendizado e pontos iniciais.

### Solução

Para resolver esse problema, será utilizada a função de Six-Hump Camelback, que é uma função multimodal com dois mínimos globais e dois mínimos locais. A função é definida como:

$$J(x, y) = (4 - 2.1x^2 + \frac{x^4}{3})x^2 + xy + (-4 + 4y^2)y^2$$

Os mínimos globais dessa função estão localizados aproximadamente em:
- $(-0.0898, 0.7126)$
- $(0.0898, -0.7126)$

O dominio típico para essa função é $x \in [-3, 3]$ e $y \in [-2, 2]$.

O gradiente da função é:

$$\frac{\partial J}{\partial x} = 8x - 8.4x^3 + \frac{4x^3}{3} + y$$
$$\frac{\partial J}{\partial y} = x + 8y - 16y^3$$

#### Definir a função de custo multimodal

In [8]:
def six_hump_camel(x: np.ndarray) -> float:
    x1 = x[0]
    x2 = x[1]

    term1 = (4 - 2.1 * x1**2 + (x1**4) / 3) * x1**2
    term2 = x1 * x2
    term3 = (-4 + 4 * x2**2) * x2**2

    return term1 + term2 + term3

def six_hump_camel_grad(x: np.ndarray) -> np.ndarray:
    x1 = x[0]
    x2 = x[1]

    dJ_dx1 = 8 * x1 - 8.4 * x1**3 + 2 * x1**5 + x2
    dJ_dx2 = x1 - 8 * x2 + 16 * x2**3

    return np.array([dJ_dx1, dJ_dx2], dtype=np.float64)

#### Calcular o ponto mínimo usando gradiente descendente para diferentes taxas de aprendizado e pontos iniciais

In [9]:
xs = [[1.0, 1.0], [2, -0.5], [-1.11, 0.8]]
learning_rates = [0.1, 0.01, 0.2]
max_iter = 1000

dict_of_results = {tuple(x): {} for x in xs}

for x in xs:
    x_tuple = tuple(x)
    for learning_rate in learning_rates:
        x_values, costs, num_iter = gradient_descendent(x, six_hump_camel, six_hump_camel_grad, learning_rate, max_iter, tolerance, 1)
        x_values = np.array(x_values)
        costs = np.array(costs)

        dict_of_results[x_tuple][learning_rate] = (x_values, costs, num_iter)

df = pd.DataFrame(columns=["Initial Point", "Learning Rate", "Final Point", "Final Cost", "Iterations"])
result_rows = []
for initial_point, results in dict_of_results.items():
    for learning_rate, (x_values, costs, num_iter) in results.items():
        final_point = x_values[-1]
        final_cost = costs[-1]
        result_rows.append({
            "Initial Point": initial_point,
            "Learning Rate": learning_rate,
            "Final Point": final_point,
            "Final Cost": final_cost,
            "Iterations": num_iter
        })

df = pd.DataFrame(result_rows)
df

C:\Users\mathe\AppData\Local\Temp\ipykernel_2196\1470203942.py:5: RuntimeWarning:

overflow encountered in scalar power

C:\Users\mathe\AppData\Local\Temp\ipykernel_2196\1470203942.py:5: RuntimeWarning:

invalid value encountered in scalar add

C:\Users\mathe\AppData\Local\Temp\ipykernel_2196\1470203942.py:6: RuntimeWarning:

overflow encountered in scalar multiply

C:\Users\mathe\AppData\Local\Temp\ipykernel_2196\1470203942.py:7: RuntimeWarning:

overflow encountered in scalar power

C:\Users\mathe\AppData\Local\Temp\ipykernel_2196\1470203942.py:15: RuntimeWarning:

overflow encountered in scalar power

C:\Users\mathe\AppData\Local\Temp\ipykernel_2196\1470203942.py:15: RuntimeWarning:

invalid value encountered in scalar add

C:\Users\mathe\AppData\Local\Temp\ipykernel_2196\1470203942.py:16: RuntimeWarning:

overflow encountered in scalar power



,Initial Point,Learning Rate,Final Point,Final Cost,Iterations
0,"(1.0, 1.0)",0.10,"[-0.08984200978246519, 0.7126564318726]",-1.031628,39
1,"(1.0, 1.0)",0.01,"[-0.08984189394682204, 0.7126563893185477]",-1.031628,208
2,"(1.0, 1.0)",0.20,"[0.0822347291225593, 0.12516157929315103]",-0.024433,1000
3,"(2, -0.5)",0.10,"[0.08984201715551254, -0.7126563677569582]",-1.031628,36
4,"(2, -0.5)",0.01,"[1.7036067100545753, -0.7960835393255994]",-0.215464,66
5,"(2, -0.5)",0.20,"[nan, nan]",NaN,1000
6,"(-1.11, 0.8)",0.10,"[-1.6220920163488715, 0.9094813133959436]",0.011036,1000
7,"(-1.11, 0.8)",0.01,"[-1.7036066787343205, 0.7960835588939981]",-0.215464,151
8,"(-1.11, 0.8)",0.20,"[-0.030549734630897135, -0.32720408684128544]",-0.368673,1000


#### Plotar a função de custo e o ponto mínimo encontrado para diferentes taxas de aprendizado e pontos iniciais

In [10]:
p_x_id = 2
initial_point = (xs[p_x_id][0], xs[p_x_id][1])

In [11]:
learning_rate = learning_rates[0]
surface_title = f"Superfície da Six-Hump Camel com ponto inicial {initial_point} e taxa de aprendizado {learning_rate}"
x_values, _, _ = dict_of_results[initial_point][learning_rate]

plot_six_hump_camel(six_hump_camel, x_values, title=surface_title, show_global_minima=True)

In [12]:
learning_rate = learning_rates[1]
surface_title = f"Superfície da Six-Hump Camel com ponto inicial {initial_point} e taxa de aprendizado {learning_rate}"
x_values, _, _ = dict_of_results[initial_point][learning_rate]

plot_six_hump_camel(six_hump_camel, x_values, title=surface_title)

In [13]:
learning_rate = learning_rates[2]
surface_title = f"Superfície da Six-Hump Camel com ponto inicial {initial_point} e taxa de aprendizado {learning_rate}"
x_values, _, _ = dict_of_results[initial_point][learning_rate]

plot_six_hump_camel(six_hump_camel, x_values, title=surface_title)